<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/M2-pol_tweets_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis

In [ ]:
# explainability (why did the model say it's hate speech)
!pip install eli5

In [ ]:
import pandas as pd
import numpy as np
import preprocessor as prepro # twitter prepro
import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

In [ ]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [ ]:
pol_tweets = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pol_tweets.gz')

In [ ]:
print(pol_tweets.iloc[-1]['text'])
print(pol_tweets.iloc[-1]['labels'])

In [ ]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [ ]:
# apply all prepro-pipeline to texts
pol_tweets['text_clean'] = text_prepro(pol_tweets['text'])

In [ ]:
pol_tweets.labels.value_counts().reset_index()

In [ ]:
# fixing sample imbalance
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(pol_tweets, pol_tweets['labels'])

In [ ]:
# Splitting the dataset into the Training set and Test set (since we have a new output variable)
X_train, X_test, y_train, y_test = train_test_split(pol_tweets['text_clean'], pol_tweets['labels'], test_size = 0.4, stratify=pol_tweets['labels'], random_state = 42)

In [ ]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [ ]:
pipe.fit(X_train,y_train) # fit model

In [ ]:
# evaluate model performance on training set

y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

In [ ]:
# overall weights (works only for linear models)
eli5.show_weights(pipe, top=20, target_names=['Rep.','Dem.'])

In [ ]:
y_pred = pipe.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
debate_tweets = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pres_debate_2020.gz')

In [ ]:
debate_tweets

In [ ]:
# apply all prepro-pipeline to texts
debate_tweets['text_clean'] = text_prepro(debate_tweets['tweet'])

In [ ]:
debate_tweets['class_pred'] = pipe.predict(debate_tweets['text_clean'])

In [ ]:
debate_tweets['dem_proba'] = pipe.predict_proba(debate_tweets['text_clean'])[:,1]

In [ ]:
debate_tweets.sort_values(['dem_proba'], ascending=True)['tweet'][:10]

In [ ]:
debate_tweets.sort_values(['dem_proba'], ascending=False)

## Topic modellng - what do dem/rep tweets say?

you can try out the tweetopic library (it's rather new, developed by people from Århus Uni. and I'm not sure about it so far: https://centre-for-humanities-computing.github.io/tweetopic/using_tweetopic.pipeline.html)

In [ ]:
debate_tweets['dem_proba'].hist()

In [ ]:
rep_tweets = debate_tweets[debate_tweets['dem_proba']<=0.2]

In [ ]:
# preprocess texts (we need tokens)
tokens = []

for text in nlp.pipe(rep_tweets['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in text 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [ ]:
rep_tweets['tokens'] = tokens

In [ ]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(rep_tweets['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in rep_tweets['tokens']]

In [ ]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=5, workers = 4, passes=10)

In [ ]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

In [ ]:
 # Let's Visualize
pyLDAvis.display(lda_display)

**Tunint model N topics**

We can evaluate coherence from gensim.
Other measures are available within the Octis libary: https://github.com/mind-Lab/octis

Also consider:

https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0


In [ ]:
from gensim.models import CoherenceModel

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=rep_tweets['tokens'], dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### other models

In [ ]:
from gensim.models import LsiModel, TfidfModel
from gensim.matutils import corpus2dense

In [ ]:
tfidf = TfidfModel(corpus)

In [ ]:
corpus_tfidf = tfidf[corpus]

In [ ]:
lsi = LsiModel(corpus_tfidf, num_topics=10, id2word=dictionary)

In [ ]:
lsi.print_topics()

In [ ]:
corpus_lsi = lsi[corpus_tfidf]

In [ ]:
lsi_matrix = corpus2dense(corpus_lsi, num_terms = 10)

In [ ]:
lsi_matrix.T[1]

In [ ]:
corpus_lsi[1]